A practitioner’s guide for building LR systems using a data-driven approach
---



This notebook accompanies the paper "From data to a validated score-based LR system: a practitioner’s guide". The paper offers a guideline to go from a data set to a validated Likelihood Ratio (LR) system in 8 steps. In this notebook, we illustrate each of these steps with working Python code. In the code, we build and validate an LR system for a data set of observations on the elememental concentrations of glass fragments.

Follow along in the notebook by running the code blocks pressing Shift+Enter on your keyboard.

The guideline involves creating multiple LR systems at first, and selection of the best LR system in a later step. In this paper we explore two options to compute scores and two options to transform scores to LRs in a two-by-two design, leading to four LR systems.

The 'scorer' options are:
1. Use the Manhattan distance (defined as the sum of the absolute differences)
2. Use a support vector machine, a machine learning model

The 'calibrator' options to transform scores to LRs are:
<ol type = 'A'>
<li>Use a generative approach, namely kernel density estimation</li>
<li>Use a discriminative approach, namely logistic regression</li>
</ol>

### Loading Python packages

We start by installing several Python packages. These are standard packages, and the LiR package that we developed and maintain. This package provides a collection of scripts to aid construction and evaluation of LR systems.



In [ ]:
# # Try to keep this in line with the requirements for lir itself
! pip install lir==1.2.0
! pip install scipy>=1.31
! pip install scikit-learn>=1.2
! pip install numpy>=1.22
# # Additional dependencies, already installed in Colab:
! pip install pandas seaborn==0.11.2

In [ ]:

import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.metrics.pairwise import paired_manhattan_distances
from sklearn.model_selection import GroupShuffleSplit
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

import lir
from lir.algorithms.bayeserror import ELUBBounder
from lir.algorithms.kde import KDECalibrator
from lir.algorithms.logistic_regression import LogitCalibrator
from lir.data.models import FeatureData
from lir.metrics import cllr
from lir.plotting import show
from lir.transform import as_transformer
from lir.transform.distance import AbsDiffTransformer
from lir.transform.pipeline import Pipeline


### Introduction LiR and sklearn


LiR follows the conventions of the sklearn package, which is the de facto standard in machine learning. This has the advantage that we can directly use all models defined in sklearn and that the code will look familiar to anyone used to sklearn. It has the disadvantage that naming of certain functions and objects may appear counterintuitive to those unfamiliar with sklearn. Sklearn defines objects that can 'fit' on data, and then 'transform' new data or 'predict' probabilities. LiR additionally allows to 'predict_lr'. We will encounter examples of these below.

### Loading the data

We download the glass data in the form of a csv file and convert this to a pandas dataframe.

In [ ]:
csv_url = 'https://raw.githubusercontent.com/NetherlandsForensicInstitute/elemental_composition_glass/main/duplo.csv'
raw_dataset = pd.read_csv(csv_url, delimiter=',').rename(
    columns = {'Item': 'Subject', 'Piece': 'Repeat', 'id': 'Id'})

# Step 1. Explore data




The data set we will use consists of elemental concentrations of float glass, obtained with LA-ICPMS (a type of mass spectrometer) ([see GitHub repository](https://github.com/NetherlandsForensicInstitute/elemental_composition_glass)). The elemental concentrations are on a log<sub>10</sub> basis, and normalized to the element silicon (Si). For each glass source, which we refer to as *subject*, two measurements are performed. Each time the concentration of ten elements is measured.

First, we will explore the type of data we have.

In [ ]:
raw_dataset.info()

As can be seen there are 13 columns for each observation:
- Id: a unique integer for each observation
- Subject: a unique integer for each subject
- Repeat: for each subject, a unique integer representing the repeated observations on this subject
- K39, Ti49, Mn55, Rb85, Sr88, Zr90, Ba137, La139, Ce140, Pb208: the processed concentration of the ten elements. The number behind the element in the column name specifies which isotope of the element was measured. The processed concentrations are all float numbers.

Furthermore, we see that there are no non-null values. In other words, there are no missing values.

Next, we have a look at the amount of data we have:

In [ ]:
print('Number of observations: ', raw_dataset['Id'].nunique())
print('Number of subjects: ', raw_dataset['Subject'].nunique())
print('Maximum number of repeated observations on one subject: ',
      raw_dataset['Repeat'].nunique())

Above you can see the number of glass panes (=distinct sources) and observations.

In this guide, we limit the number of items to ensure that we do not run out of RAM during the subsequent calculations.

In [ ]:
# Make sure that the sorting is first by item, then by repeat
raw_dataset = raw_dataset.sort_values(by=['Subject', 'Repeat'], axis=0, ascending=True)

# Only keep the first 400 rows, as the free version of Google Colab does not
# have enough RAM available for computations on the full set
raw_dataset = raw_dataset.drop(raw_dataset.index[400:])

As a check, the code block above can be run again, now showing 400 observations.

Each row of the dataframe contains one observation:


In [ ]:
print(raw_dataset.head())



To get a bit more feeling for the data, we create a histogram of all observations per element together:


In [ ]:
print('Histogram of all measured concentrations:')
raw_dataset[['K39', 'Ti49', 'Mn55', 'Rb85', 'Sr88', 'Zr90', 'Ba137', 'La139',
          'Ce140', 'Pb208']].hist(figsize=(10,10), layout=(2,5));

In the histograms, we see that there are no distinctive outliers, although both Mn55 and Ba137 show data points with a relatively high value of around 4. As these is no reason to suspect an error, these probably reflect variability in the population and should not be removed.

Now, we select the variables (the 10 elements) and labels (indicating which subject was measured) and convert them to a numpy array. This is the format needed for the rest of the steps.

In [ ]:
variables = ['K39', 'Ti49', 'Mn55', 'Rb85', 'Sr88', 'Zr90', 'Ba137', 'La139',
             'Ce140', 'Pb208']
labels = ['Subject']

observations = raw_dataset[variables].to_numpy()
ids = raw_dataset[labels].to_numpy()

### Explore data - a deeper look

Looking at the data more closely, we see that the concentrations for repeat measurements on the same subject are more similar to each other than the concentrations measured for different subjects. To examine this further, we calculate for each subject the standard deviation of its two measurements (a measure of within-source variability). We compare this to the standard deviation of the mean concentrations across all subjects (a measure of between-source variability). We expect the former to be smaller.

In [ ]:
print('Histograms of standard deviations for the repeat measurements on a subject:')
raw_dataset.groupby('Subject')[variables].std().hist(figsize=(10,10), layout=(2,5));

In [ ]:
print('Between subject standard deviations compared to the maximum within subject standard deviation:')
st_devs = pd.DataFrame(raw_dataset.groupby('Subject')[variables].mean().std())
st_devs.columns = ['std_between']
st_devs['std_within_mean'] = pd.DataFrame(
    raw_dataset.groupby('Subject')[variables].std().mean())
st_devs['std_within_max'] = pd.DataFrame(
    raw_dataset.groupby('Subject')[variables].std().max())
st_devs

We see that the within-source standard deviations roughly lie between 0 and 0.03 (second column), and between-source standard deviations are much larger (first column). This holds for all elements. In fact, the between objects standard deviation is larger than the largest within standard deviation we encounter (third column). This indicates that an LR system based on this data can be discriminative.

When different variables are considered for calculating an evidential value, it is best when these variables have a correlation coefficient close to zero. If two variables are highly correlated however, they can still both be used in the LR system; their combined within variance is reduced compared to measuring a single variable. If the number of variables is large compared to the number of subjects, we may consider dimension reduction.

To see to what extent the concentrations of the different elements are correlated, we create scatterplots plotting the concentrations against each other.

In [ ]:
sns.set_theme(style='ticks')
sns.pairplot(raw_dataset.reset_index()[variables])
plt.show()
plt.close()

We see that some of the values are correlated, for example for K39 and Rb85. Most scatterplots seem quite random, indicating a low correlation. This is indeed reflected in the correlation matrix:

In [ ]:
corr = raw_dataset[variables].corr()
corr.style.background_gradient()

As the level of correlation is not too high, in this notebook we proceed with these data as they are. However, given that some variables are (strongly) correlated, we encourage the reader to experiment with improving the resulting LR system by handling this correlation. For example, by combining these variables or removing some from our data set (a form of dimensionality reduction), or utilizing models that allow for high correlation between variables.

# Step 2. Split data in different subsets


As explained in the manuscript, splitting the data is a necessary step in order to evaluate the performance of the model while avoiding overfitting. This means that part of the data will be used to build the model (training and selection sets) and another part is reserved for validation.

As shown in the steps below, all three sets should be independent, e.g. consist of disjoint sets of glass objects. First, we set 20% of the objects aside for the validation set, and then we split the remaining objects into a training (80%) and a selection (20%) set.

```
Total: 400
        / \
      320  80 (validation)
      / \
    256  64
 (train) (test)
```

In [ ]:
# First we split off 20% from the data as a hold-out validation set (grouped per glass fragment)
splitter = GroupShuffleSplit(test_size=.20, n_splits=2, random_state=1)
split = splitter.split(observations, groups=ids)  # train (80%), test (20%)

test_train_indices, validation_pairs = next(split)

# Then we split off 20% to use as a test set from the remaining 80%
splitter = GroupShuffleSplit(test_size=.20, n_splits=2, random_state=1)
split = splitter.split(observations[test_train_indices], groups=ids[test_train_indices])
train_indices, test_indices = next(split)

# We create the train, test (or 'selection') and validation set
# lir's FeatureData class is used to hold the features and labels together
train = FeatureData(features=observations[train_indices], labels=ids[train_indices].flatten())
test = FeatureData(features=observations[test_indices], labels=ids[test_indices].flatten())
validation = FeatureData(features=observations[validation_pairs], labels=ids[validation_pairs].flatten())

print(f'Size of total data set:  {len(observations):<4} (100%)')
print(f'Size of training set:    {len(train):<4} (64% (80% of 80%))')
print(f'Size of test set:        {len(test):<4} (16% (20% of 80%))')
print(f'Size of validation set:  {len(validation):<4} (20%)')

# Step 3. Pre-process data

This step entails transformation of the raw data to informative features that can be used by a statistical model. Standard transformations for concentration data are taking the log<sub>10</sub> and normalising by a certain variable (in this particular example: dividing all other variables by the value of Si), both of which are already applied per observation to the glass data we use.

To show another example of preprocessing, in addition we here apply a z-score transformation per column to the data set: subtracting the mean and dividing by the standard deviation.

In [ ]:

print('The first row of observations\n')
print(f'   - before transformation: \n{train.features[0]}\n')

# The `as_transformer` function wraps the `StandardScaler` class from scikitlearn
# within a class that support lir's `FeatureData` objects.
z_score_transformer = as_transformer(StandardScaler())

# lir transformers follow the fit/transform paradigm.
# It can be fit and transformed seperately, or in one go using `fit_transform`.
train_zscore = z_score_transformer.fit_transform(train)
test_zscore = z_score_transformer.fit_transform(test)
validation_zscore = z_score_transformer.fit_transform(validation)

print(f'    - after transformation: \n{train_zscore.features[0]}')

We can see what this transformation does for one of the columns, that of the element K39, in the figure below. The distribution is scaled and shifted, but otherwise the same. In a plot, this manifests itself as adjusted x-values:

In [ ]:
# We use `[:, 0]` to select the first feature (K39) for visualization.
plt.hist(train.features[:, 0], bins=20, histtype='step', color='tab:purple', alpha=1, label='before pre-processing')
plt.hist(train_zscore.features[:, 0], bins=20, color='tab:cyan', alpha=0.5, label='after pre-processing')
plt.xlabel('K39 value')
plt.ylabel('count')
plt.legend()

# Step 4. Calculate scores

In the previous steps, we have explored the given data, split it into subsets and performed pre-processing. We call the 10 pre-processed values we now have for each observation the features. The next step is to go from these feature vectors to a score. In order to do this we have to make pairs of observations, both *H*<sub>1</sub>-true and *H*<sub>2</sub>-true, and calculate the scores based on their feature vectors. A score is meant to quantify the degree of (dis)similarity between a pair of observations.


### Create pairs

We create the pairs for the subjects in the training data and the selection data.

In [ ]:
from lir.transform.pairing import InstancePairing


instance_pairer = InstancePairing(ratio_limit=2)

# The InstancePairing transformer creates all pairs. This transforms the
# FeatureData object into a PairedFeatureData object. This object holds
# both the pairs of observations and the corresponding hypothesis labels, and
# is a subclass of FeatureData.
train_pairs = instance_pairer.pair(train_zscore)
test_pairs = instance_pairer.pair(test_zscore)
validation_pairs = instance_pairer.pair(validation_zscore)

print(f'We have constructed {len(train_pairs)} pairs in the training data, \
{int(sum(train_pairs.labels==1))} H1 and {int(sum(train_pairs.labels==0))} H0/H2 pairs.')

print('Looking at the values for the first pair, \n')
print(train_pairs.features[0])
print('\nwe see this is just the concatenation of the feature vectors of the \
two observations on the first subject:\n')
print(train_zscore.features[:2,])

Once we have the same-source and different-source pairs we have to compute the score per pair. We show two different ways.

### Option A: Compute (dis)similarity score
We take the Manhattan distance. The Manhattan distance is the sum of the absolute differences of the features. Luckily, distances such as these are standard functions in the python package sklearn.

In [ ]:
print(train_pairs.features[:,0])

In [ ]:
manhatten_score_train = FeatureData(
    features=paired_manhattan_distances(train_pairs.features[:,0], train_pairs.features[:,1]),
    labels=train_pairs.labels)
manhatten_score_select = FeatureData(
    features=paired_manhattan_distances(test_pairs.features[:,0], test_pairs.features[:,1]),
    labels=test_pairs.labels)



manhatten_scores_train_h1 = manhatten_score_train.features[train_pairs.labels==1]
manhatten_scores_train_h2 = manhatten_score_train.features[train_pairs.labels==0]


print("We first examine a h1 (pair from the same subject) and a h2 (pair from different subjects) pair's distances:")
print('- For the first h1 pair, the manhatten distance is: ', manhatten_scores_train_h1[0])
print('- For the first h2 pair, the manhatten distance is: ', manhatten_scores_train_h2[0])

More generally, we see that *H*<sub>2</sub>-true pairs get higher dissimilarity scores, as expected:

In [ ]:
with show() as ax:
    ax.score_distribution(scores=manhatten_score_train.features, y=manhatten_score_train.labels,
                          bins=np.linspace(0, 30, 60), weighted=True)
    plt.xlabel('dissimilarity score')
    H1_legend = mpatches.Patch(color='tab:blue', alpha=.3, label='$H_1$-true')
    H2_legend = mpatches.Patch(color='tab:orange', alpha=.3, label='$H_2$-true')
    ax.legend(handles=[H1_legend, H2_legend])

### Option B: Compute machine learning score
Alternatively, we can fit a statistical model to compute scores. In this case, we use a support vector machine. First, the absolute differences for each elemental value between the pairs of observations are computed. The support vector machine then assigns a score to each pair. The model aims to assign high scores to *H*<sub>1</sub>-true pairs and low scores to *H*<sub>2</sub>-true pairs.

In [ ]:
# Machine learning models need a single vector as input. The AbsDiffTransformer takes two feature vectors,
# one for each subject of a pair, and returns the elementwise absolute differences.
# The AbsDiffTransformer and support vector machine (SVC) are combined into a single pipeline using sklearns Pipeline class.
machine_learning_scorer = Pipeline([('abs_difference', AbsDiffTransformer()), ('classifier', SVC(probability=True))])

# The model has to be fit on the data
machine_learning_scorer.fit(train_pairs)
# Scores can be computed using the 'predict_proba' function. This is another sklearn convention,
# which returns two columns of which we take the second using '[:,1]'
machine_learning_scores_train = machine_learning_scorer.transform(train_pairs)

The scores look like this:

In [ ]:
with show() as ax:
    ax.score_distribution(scores=machine_learning_scores_train.features, y=train_pairs.labels,
                          bins=np.linspace(0, 1, 10), weighted=True)
    plt.xlabel('machine learning score')
    H1_legend = mpatches.Patch(color='tab:blue', alpha=.3, label='$H_1$-true')
    H2_legend = mpatches.Patch(color='tab:orange', alpha=.3, label='$H_2$-true')
    ax.legend(handles=[H1_legend, H2_legend])

In [ ]:
print('For example, the average machine learning scores are')
print(f'H1-true pairs {np.average(machine_learning_scores_train.features[train_pairs.labels==1]):.4f}. ')
print(f'H2-true pairs {np.average(machine_learning_scores_train.features[train_pairs.labels==0]):.4f}. ')

As expected, the *H*<sub>1</sub>-true pairs generally have lower dissimilarity scores than the *H*<sub>2</sub>-true pairs, while their machine learning scores are higher.

# Step 5. Calculate LRs from scores

After step 4 we have a score for each pair. Two approaches exist to convert these scores to LRs. We show an example for each approach.



### Generative approach

In the generative approach, we model the distribution of scores under *H*<sub>1</sub> and under *H*<sub>2</sub>. Here, we use kernel density estimation (KDE) to do this. KDE is a way to estimate an unknown probability density by 'smoothing' the observed distribution. LiR defines classes called 'Calibrators' to perform the score to LR mapping.

In [ ]:
kde_calibrator =  KDECalibrator(bandwidth='silverman')

To illustrate how this works, we plot the histograms for the scores under *H*<sub>1</sub> and *H*<sub>2</sub>-true, together with the KDE fits.


In [ ]:
kde_calibrator.fit(manhatten_score_train.features, manhatten_score_train.labels)
with show() as ax:
    ax.calibrator_fit(kde_calibrator, score_range=[0,20])
    plt.xlabel('dissimilarity score')
    plt.ylabel('likelihood ratio')
    plt.title('KDE Calibrator fit on manhatten distance scores')
    

In [ ]:
kde_calibrator.fit(manhatten_score_train.features, manhatten_score_train.labels==1)
with show() as ax:
    ax.calibrator_fit(kde_calibrator, score_range=[0, 30])
    ax.score_distribution(scores=manhatten_score_train.features, y=(manhatten_score_train.labels==1)*1,
                          bins=np.linspace(0, 30, 60), weighted=True)
    ax.xlabel('dissimilarity score')
    H1_legend = mpatches.Patch(color='tab:blue', alpha=.3, label='$H_1$-true')
    H2_legend = mpatches.Patch(color='tab:orange', alpha=.3, label='$H_2$-true')
    ax.legend(handles=[H1_legend, H2_legend])

### Discriminative approach
Alternatively, we can use other discriminative methods. Here we use logistic regression to map the score to posterior odds. This can be corrected for the amount of *H*<sub>1</sub> and *H*<sub>2</sub> data.

In [ ]:
logreg_calibrator = LogitCalibrator()

### Bounding the LRs

The LR system may output values that seem too large or too small given the size of our data set. Several methods have been proposed mitigate this problem, by 'shrinking' the LRs towards 1. Here we show how to use empirical lower and upper bounds (ELUB) to limit the minimum and maximum values the LR system can output. Using LiR, we can stack this 'ELUBbounder' on any calibrator we have.

In [ ]:
bounded_kde_calibrator = Pipeline([('kde_calibrator', kde_calibrator), ('elub', ELUBBounder())])
bounded_logreg_calibrator = Pipeline([('logreg_calibrator', logreg_calibrator), ('elub', ELUBBounder())])

### Illustration of scores to LRs
To illustrate the score to LR mapping, we plot how the dissimilarity scores are mapped to LRs (using a log<sub>10</sub> scale). We print the values of the ELUB bounds - these are visible in the plot as the minimum and maximum values the LRs obtain (horizontal lines).

One thing to note is that the purple (logistic regression) never goes up. This is a desirable property, it makes sense that more extreme scores correspond to more extreme LRs. In contrast, the cyan line (kernel density estimation (KDE)) shows several humps, e.g. at score=0 and score=2. This means that, counterintuitively, sometimes a higher dissimilarity leads to a lower LR. Here we don't 'fix' this (e.g. by adjusting the bandwidth of the KDE), but be aware that some methods for transforming scores to LRs can have this property.

In [ ]:
lrs_train_logreg = bounded_logreg_calibrator.fit_transform(manhatten_score_train)
lrs_train_kde = bounded_kde_calibrator.fit_transform(manhatten_score_train)

plt.scatter(manhatten_score_train.features, lrs_train_logreg.llrs, color='tab:purple', marker='.', label='logistic regression')
plt.scatter(manhatten_score_train.features, lrs_train_kde.llrs, color='tab:cyan', label = 'KDE')
plt.xlim([0, 5])

plt.legend()

plt.xlabel('dissimilarity score')
plt.ylabel('log$_{10}$(LR)')

Note that for the generative approach, the score to LR function is not monotonous. This is something we do not expect from a score to LR function, and possibly could be solved by using a kernel density estimator with a greater bandwidth.

# Step 6. Select best LR system



In this step, we compute the performance of each LR system defined on the selection data. For this, we use the CalibratedScorer class from LiR, which allows you to combine computing scores and transforming to LRs in one step. A CalibratedScorer needs two parts: a function or machine learning model to compute scores, and a Calibrator to transform the scores to LRs.

We define a function to give us the histograms of the LRs, the PAV plot, the ELUB bounds and the *C*<sub>llr</sub>:

In [ ]:
from lir.data.models import LLRData


def show_performance(lrs: LLRData):

    # LR histogram
    with show() as ax:
        ax.lr_histogram(lrs)
        ax.title('Histogram of the LRs')
        H1_legend = mpatches.Patch(color='tab:blue', alpha=.25, label='$H_1$-true')
        H2_legend = mpatches.Patch(color='tab:orange', alpha=.25, label='$H_2$-true')
        ax.legend(handles=[H1_legend, H2_legend])

    # print(f'\nELUB log(LR) bounds are {np.log10(calibrator._lower_lr_bound):.2f} and\
    # {np.log10(calibrator._upper_lr_bound):.2f} \n')

    print('PAV plot (closer to the line y=x is better):\n')
    # Show the PAV plot (closer to the line y=x is better)
    with show() as ax:
        ax.pav(lrs)

    # Print the quality of the system as log likelihood ratio cost (lower is better)
    print(f'\n The log likelihood ratio cost (Cllr) is {cllr(lrs.llrs, lrs.labels):.3f} (lower is better)\n')

### Option 1: Manhattan distance + KDE


In [ ]:
# We define the CalibratedScorer. We use the standard sklearn function
# paired_manhattan_distance to compute the scores, and an ELUB bounded
# KDE calibrator to transform to LRs.
# lr_system = lir.CalibratedScorer(paired_manhattan_distances, bounded_kde_calibrator)
lr_system = Pipeline([('calibrator', bounded_kde_calibrator)])
# We fit the entire LR system
lr_system.fit(manhatten_score_train)

# We compute the LRs on the selection data
lrs_select = lr_system.transform(manhatten_score_select)

# This is how well the system performs
show_performance(lrs_select)

### Option 2: Manhattan distance + logistic regression

In [ ]:
# We define the CalibratedScorer. We use the standard sklearn function paired_manhattan_distance to compute the scores,
# and an ELUB bounded logistic regression calibrator to transform to LRs.
lr_system = Pipeline([('logreg_calibrator', logreg_calibrator), ('elub', ELUBBounder())])
# We fit the entire LR system
lr_system.fit(manhatten_score_train)

# We compute the LRs on the selection data
lrs_select = lr_system.transform(manhatten_score_select)

# This is how well the system performs
show_performance(lrs_select)

### Option 3: support vector machine + KDE



In [ ]:
# The whole LR system consists of computing the score and then mapping the scores to LRs.
# We use the machine learning scorer introduced in step 4
lr_system = Pipeline([('abs_difference', AbsDiffTransformer()), 
                      ('classifier', SVC(probability=True)), 
                      ('kde', kde_calibrator)
                      ])

lr_system.fit(train_pairs)
lrs_select = lr_system.transform(validation_pairs)
lrs_select = LLRData(labels=validation_pairs.labels, 
                     features=lrs_select.features,
                     llrs=lrs_select.features.flatten())
show_performance(lrs_select)

# lrs_select

### Option 4: support vector machine + logistic regression


In [ ]:
# The whole LR system consists of computing the score and then mapping the scores to LRs
lr_system = Pipeline([('abs_difference', AbsDiffTransformer()), 
                      ('classifier', SVC(probability=True)), 
                      ('logreg', LogitCalibrator()), 
                      ('elub', ELUBBounder())])

# lr_system = lir.CalibratedScorer(scorer, bounded_logreg_calibrator)
# We fit the whole system
lr_system.fit(train_pairs)
lrs_select = lr_system.transform(validation_pairs)

show_performance(lrs_select)

### Final selection
All four options show reasonable LR distributions. Because the glass data contain much information, the *H*<sub>1</sub> and *H*<sub>2</sub> are almost perfectly separated. Most of the *H*<sub>2</sub> pairs, and many of the *H*<sub>1</sub> pairs fall on the ELUB bounds. We see similar performance across the four systems. The LR distributions look reasonable for all systems, the PAV plots look good for options 2 and 3 (with the points following the line y=x) and show slightly conservative LRs for options 1 and 4 (points lying above the line y=x).

We have a slight preference for option 3, which has a good PAV plot, the best (=lowest) *C*<sub>llr</sub> and best (=widest) ELUB bounds. Option 2 would also have been a reasonable choice, particularly as the Manhattan distance is much more explainable than the support vector machine.

# Step 7. Validate selected LR system


After selecting the (best) model, you should assess the performance of the system on the validation set.

### Construct the selected system

Since we want to build the selected model based on as much data as possible, we add the selection set to the training set and use the combination to train the LR system used for validation.

In [ ]:
# Create the combined data set
# obs_train_select = observations[train_select_indices]
# ids_train_select = ids[train_select_indices]

# Step 3 pre-processing: normalise
z_score_transformer.fit(validation)
# train_zscore = z_score_transformer.transform(observations)

# Step 4: combine the pairs into one feature vector by taking the absolute difference
# obs_pairs_train_select, hypothesis_train_select = create_pairs(train_zscore, ids_train_select)
# obs_pairs_val, hypothesis_val = create_pairs(train_zscore, ids_val)

selected_lr_system = Pipeline([('abs_difference', AbsDiffTransformer()), 
                                ('classifier', SVC(probability=True)),
                                ('kde', kde_calibrator), 
                                ('elub', ELUBBounder())])

# Step 4+5 combined: we fit the whole system
selected_lr_system.fit(train_pairs)

# Compute the LRs on the validation data
lrs_val = selected_lr_system.transform(validation_pairs)

### Calculate system performance

In [ ]:
# We always inspect the characteristics we also look at in selection
show_performance(lrs_val)

In [ ]:
# There are many other characteristics that we may want to inspect,
# such as the empirical cross entropy (ECE) plot
with show() as ax:
      ax.ece(lrs_val)

In the validation, we hope to see performance comparable to that on the selection data. This is the case here, with *C*<sub>llr</sub>, ELUB bounds and LR distributions looking very similar to those we saw in step 6. The PAV plot looks reasonable, with the data points mostly following the line y=x. There are a few datapoints that lie under this line, indicating a slight overconfidence of the system in this LR range. The ECE plot in Figure shows that the LRs are, for any prior odds, performing substantially better than a system where LR=1 (the reference system).

# Step 8. Construct casework LR system


To obtain our casework LR system, we fit on all available data:

In [ ]:
# Create new pairs for the combined data

full_dataset = FeatureData(features=observations, labels=ids.flatten())
full_zscore = z_score_transformer.fit_transform(full_dataset)
full_pairs = instance_pairer.pair(full_zscore)
# obs_pairs, hypothesis = create_pairs(train_zscore, ids)

# Fit the same system on all the data
selected_lr_system.fit(full_pairs);

Note that we cannot empirically test this system. In casework, if you have decided to use the system, you would input your pair of observation and get back an LR.

In [ ]:
observation_1 = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
observation_2 = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

observations_12 = np.array([[observation_1, observation_2]])

observations_12_fd = FeatureData(features=observations_12)


print(f'The LL  R obtained is {selected_lr_system.transform(observations_12_fd).llrs}')